In [2]:
from collections import defaultdict
import sqlite3
import numpy as np

In [91]:
# INPUTS
query_file = "random_query.fa"  # random query selected as a slice from seq: seq[100:400] len=300
probability_file = "chr22.maf.ancestors.42000000.complete.boreo.conf.txt"
fasta_file = "chr22.maf.ancestors.42000000.complete.boreo.fa.txt"

# PARAMS
w = 11
MATCH_SCORE = 1
MISMATCH_SCORE = -1
M = np.array((4, 4))
delta = 10
alpha = 0.8*w
beta = 100


In [4]:
# Open the probability file and store it is as a list of floats
with open(probability_file, 'r') as f:
    prob = f.readline()
prob = prob.split(' ')
prob = prob[:-1]
prob = [float(i) for i in prob]

# Open the fasta file and store it as a string
with open(fasta_file, 'r') as f:
    db = f.readline()

# Open the query file and store it as a string
with open(query_file) as f:
    query = f.readline()
    
# Test import
print(db[:10])
print(prob[:10])
print(query[:10])

CAACTAACCA
[0.99, 1.0, 1.0, 0.99, 0.98, 1.0, 1.0, 0.99, 0.99, 1.0]
CATCAACCAC


In [5]:
conn = sqlite3.connect('blast.db')
c = conn.cursor()

In [6]:
def get_table_name(word_size):
    return "preprocess_wordsize_" + str(word_size)

In [7]:
nucleotide_num = {'A':0,'C':1,'G':2,'T':3}

def get_word_encoding(word):
    inverted_word = word[::-1]
    n = len(word)
    
    index = 0
    
    for i in range(n):
        index += pow(4,i) * nucleotide_num[inverted_word[i]] 
    
    return index + 1

In [8]:
def get_indexes_for_word(word):
    word_size = (len(word))
    new_table_name = get_table_name(word_size)
    
    encoding = get_word_encoding(word)
    
    s = "SELECT sequence_index FROM {} where word_encoding = ?".format(new_table_name)
    return c.execute(s,(encoding,))

In [9]:
def singleBaseCompare(base1, base2):
    if base1 == base2:
        return MATCH_SCORE
    else:
        return MISMATCH_SCORE

In [45]:
def scoreSeed(index):
    """ Given the starting index in the database of a seed
    returns the score of that seed based on the product of MATCH_SCORE and probabilities"""
    seed_score = 0
    for i in range(index, index + w + 1):
        seed_score += prob[i] * MATCH_SCORE
    return seed_score

In [11]:
# Get all possible words from query and store them in the list of strings words
words = []
i = 0
while(i+w <= len(query)):
    words.append(query[i:i+w])
    i += 1

words[:3]

['CATCAACCACA', 'ATCAACCACAG', 'TCAACCACAGA']

In [89]:
def ungappedExtensionRight(query_index, db_index, seed_score):
    """Takes the index of the query and db at the end the seed and the seed_score
    outputs the indices of the ungapped extension and its score"""
    max_score = seed_score
    maxscoring_qi = 0
    maxscoring_dbi = 0
    score = seed_score
    query_index += 1
    db_index += 1
    
    # While loop that exits when the difference between max_score acheived and score is greater than delta
    while max_score - score < delta and query_index < len(query) and db_index < len(db):
        score += singleBaseCompare(query[query_index], db[db_index])
        if score > max_score:
            max_score = score
            maxscoring_qi = query_index
            maxscoring_dbi = db_index
        query_index += 1
        db_index += 1
        
    
    return (maxscoring_qi, maxscoring_dbi, max_score)

def ungappedExtensionLeft(query_index, db_index, seed_score):
    """Takes the index of the query and db at the start the seed and the seed_score
    outputs the indices of the ungapped extension and its score"""
    max_score = seed_score
    maxscoring_qi = 0
    maxscoring_dbi = 0
    score = seed_score
    
    # While loop that exits when the difference between max_score acheived and score is greater than delta
    while max_score - score < delta and query_index >= 0 and db_index >= 0:
        query_index -= 1
        db_index -= 1
        score += singleBaseCompare(query[query_index], db[db_index])
        if score > max_score:
            max_score = score
            maxscoring_qi = query_index
            maxscoring_dbi = db_index
    
    return (maxscoring_qi, maxscoring_dbi, max_score)

In [90]:
# Putting it all together
qi = 0
total_seeds = 0
stop_before_ungap = 0
stop_before_gap = 0
HSPs = 0
for word in words:
    cursor = get_indexes_for_word(word)
    pos_list = cursor.fetchall()
    
    for pos in pos_list:
        total_seeds += 1
        
        # Score Seed
        seed_score = scoreSeed(pos[0])
        if seed_score < alpha:
            stop_before_ungap += 1
            print(word + " at pos " + str(pos[0]) + " stopped before \t ungapped Extension \t (seed score: "+str(seed_score)+" )")
            continue
        
        # ungapped Extension
        right = ungappedExtensionRight(qi+w, pos[0]+w, seed_score)
        left = ungappedExtensionLeft(qi, pos[0], seed_score)
        HSP_score = right[2] + left[2] + seed_score
        if HSP_score < beta:
            stop_before_gap += 1
            print(word + " at pos " + str(pos[0]) + " stopped before \t gapped Extension \t (HSP score: "+str(HSP_score)+")")
            continue
            
        HSPs += 1
            
    qi += 1
    
    
print("total_seeds: ", total_seeds)  
print("stopped before ungapped extension: ", stop_before_ungap)
print("stopped before gapped extension: ", stop_before_gap)
print("HSPs: ", HSPs)


CCACAGAGTCT at pos 233022 stopped before 	 gapped Extension 	 (HSP score: 36.64)
ACAGAGTCTGG at pos 561923 stopped before 	 gapped Extension 	 (HSP score: 33.660000000000004)
CAGAGTCTGGG at pos 408422 stopped before 	 gapped Extension 	 (HSP score: 34.53)
CAGAGTCTGGG at pos 561924 stopped before 	 gapped Extension 	 (HSP score: 33.31)
AGAGTCTGGGG at pos 207498 stopped before 	 gapped Extension 	 (HSP score: 33.78)
GAGTCTGGGGG at pos 207499 stopped before 	 gapped Extension 	 (HSP score: 34.720000000000006)
GAGTCTGGGGG at pos 355060 stopped before 	 gapped Extension 	 (HSP score: 37.370000000000005)
AGTCTGGGGGC at pos 396084 stopped before 	 gapped Extension 	 (HSP score: 32.07000000000001)
GTCTGGGGGCC at pos 396085 stopped before 	 gapped Extension 	 (HSP score: 33.010000000000005)
GTCTGGGGGCC at pos 454391 stopped before 	 gapped Extension 	 (HSP score: 35.07000000000001)
TCTGGGGGCCC at pos 511872 stopped before 	 gapped Extension 	 (HSP score: 34.29)
CTGGGGGCCCA at pos 396352 stopped

TTTTTTTTTGA at pos 44474 stopped before 	 gapped Extension 	 (HSP score: 28.419999999999995)
TTTTTTTTTGA at pos 117814 stopped before 	 gapped Extension 	 (HSP score: 35.55)
TTTTTTTTTGA at pos 198561 stopped before 	 gapped Extension 	 (HSP score: 32.7)
TTTTTTTTTGA at pos 250543 stopped before 	 gapped Extension 	 (HSP score: 35.87)
TTTTTTTTTGA at pos 339507 stopped before 	 gapped Extension 	 (HSP score: 42.84)
TTTTTTTTTGA at pos 388268 stopped before 	 ungapped Extension 	 (seed score: 7.96 )
TTTTTTTTGAG at pos 16545 stopped before 	 gapped Extension 	 (HSP score: 34.18000000000001)
TTTTTTTTGAG at pos 198562 stopped before 	 gapped Extension 	 (HSP score: 33.38)
TTTTTTTTGAG at pos 250544 stopped before 	 gapped Extension 	 (HSP score: 35.93)
TTTTTTTTGAG at pos 339508 stopped before 	 gapped Extension 	 (HSP score: 41.40000000000001)
TTTTTTTGAGA at pos 16546 stopped before 	 gapped Extension 	 (HSP score: 34.78999999999999)
TTTTTTTGAGA at pos 250545 stopped before 	 gapped Extension 	

AGCCTCCATGG at pos 315 stopped before 	 ungapped Extension 	 (seed score: 8.49 )
AGCCTCCATGG at pos 417703 stopped before 	 gapped Extension 	 (HSP score: 30.939999999999998)
GCCTCCATGGT at pos 541509 stopped before 	 gapped Extension 	 (HSP score: 30.160000000000004)
GGTTCAAGTGA at pos 198647 stopped before 	 gapped Extension 	 (HSP score: 45.35000000000001)
GGTTCAAGTGA at pos 533083 stopped before 	 gapped Extension 	 (HSP score: 34.46)
GTTCAAGTGAT at pos 198648 stopped before 	 gapped Extension 	 (HSP score: 47.739999999999995)
GTTCAAGTGAT at pos 533084 stopped before 	 gapped Extension 	 (HSP score: 34.43)
TTCAAGTGATT at pos 326 stopped before 	 ungapped Extension 	 (seed score: 8.62 )
TTCAAGTGATT at pos 23931 stopped before 	 gapped Extension 	 (HSP score: 33.900000000000006)
TTCAAGTGATT at pos 533085 stopped before 	 gapped Extension 	 (HSP score: 34.459999999999994)
TCAAGTGATTC at pos 23932 stopped before 	 gapped Extension 	 (HSP score: 35.93)
TCAAGTGATTC at pos 51374 stopped b

CAGCCTCCCAG at pos 42700 stopped before 	 gapped Extension 	 (HSP score: 36.35)
AGCCTCCCAGC at pos 42701 stopped before 	 gapped Extension 	 (HSP score: 35.63)
GCCTCCCAGCA at pos 42702 stopped before 	 gapped Extension 	 (HSP score: 35.57000000000001)
GCCTCCCAGCA at pos 530759 stopped before 	 gapped Extension 	 (HSP score: 34.48)
CCTCCCAGCAG at pos 42703 stopped before 	 gapped Extension 	 (HSP score: 36.57000000000001)
CCTCCCAGCAG at pos 193897 stopped before 	 gapped Extension 	 (HSP score: 33.49000000000001)
CCTCCCAGCAG at pos 206985 stopped before 	 gapped Extension 	 (HSP score: 36.13)
CCTCCCAGCAG at pos 591753 stopped before 	 gapped Extension 	 (HSP score: 34.050000000000004)
CTCCCAGCAGT at pos 591754 stopped before 	 gapped Extension 	 (HSP score: 35.02)
TCCCAGCAGTA at pos 351 stopped before 	 ungapped Extension 	 (seed score: 8.6 )
CCCAGCAGTAG at pos 352 stopped before 	 ungapped Extension 	 (seed score: 8.76 )
CCAGCAGTAGC at pos 353 stopped before 	 ungapped Extension 	 (see

In [55]:
len(query)

300

In [ ]:
conn.close()